<a href="https://colab.research.google.com/github/vibhorjoshi/kaggle-challenge/blob/main/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:








!pip install nltk

import nltk # imports the nltk module
nltk.download() # calls the download function from the nltk modulel

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_eng Averaged Perceptron Tagger (JSON)
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] averaged_perceptron_tagger_rus Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] bcp47............... BCP-47 Language Tags
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ 

True

In [ ]:
pip install optuna

In [ ]:
luxurious_brands = [
    "Bentley",
    "Maserati",
    "Lamborghini",
    "Rolls-Royce",
    "Ferrari",
    "McLaren",
    "Aston",
    "Maybach",
]


Top_10_brands = [
    "Tesla",
    "Toyota",
    "BYD",
    "Ferrari",
    "Mercedes-Benz",
    "Porsche",
    "BMW",
    "Xiaomi",
    "Volkswagen",
    "Honda",
]

pattern_v = ["I4", "V6", "V8", "V10", "V12"]

pattern = [
    "A/T",
    "w/Dual",
    "M/T",
    "Automatic",
    "CVT",
    "Auto-Shift",
    "Manual",
    "Variable",
    "F",
    "Overdrive",
    "Electronically",
    "At/Mt",
    "DCT",
    "CVT-F",
    "AT",
]


def preproc(df):
    df.replace("–", np.nan, inplace=True)
    pattern_ex = r"(?:^|\s+)(" + "|".join(pattern) + r")(?:\s+|$)"
    pattern_vex = r"(?:^|\s+)(" + "|".join(pattern_v) + r")(?:\s+|$)"
    df.replace("–", np.nan, inplace=True)
    df["transm_"] = df["transmission"].str.extract(pattern_ex, expand=False)
    df["hp"] = df["engine"].str.extract(r"(\d+\.\d+)HP", expand=False)
    df["cyl"] = df["engine"].str.extract("(L.*(?=.Cyl))")
    df["cyl"] = np.where(
        df["cyl"].isnull(),
        df["engine"].str.extract(pattern_vex, expand=False),
        df["cyl"],
    )
    df["cyl"] = df["cyl"].str.extract(r"(\d*\.?\d+)")
    df["engine"] = df["engine"].replace(" Liter", "L", regex=True)
    df["liter"] = df["engine"].str.extract(r"(\d+\.\d+)L", expand=False)
    df["accident"] = df["accident"].replace(
        {"None reported": 0, "At least 1 accident or damage reported": 1}
    )
    df["luxurious_brands"] = np.where(
        df["brand"].isin(luxurious_brands), "luxury", "regular"
    )
    df["top_10_brands"] = np.where(
        df["brand"].isin(luxurious_brands), "top_10", "unranked"
    )
    df["rank"] = df['milage'].astype(str) + '_' + df['hp'].astype(str) + '_' + df['cyl'].astype(str)
    df["rank"] = df["rank"].rank(method='dense')
    df = df.drop(["transmission", "engine"], axis=1)
    list_str_obj_cols = df.columns[df.dtypes == "object"].tolist()
    df[list_str_obj_cols] = df[list_str_obj_cols].astype("category")
    df[["cyl", "accident"]] = df[["cyl", "accident"]].astype("category")
    df[["hp", "liter"]] = df[["hp", "liter"]].astype("float16")
    return df

In [ ]:
train_df=pd.read_csv("/content/train.csv")
test_df=pd.read_csv("/content/test.csv")

In [ ]:
train_df_for_model = preproc(train_df)
train_df_for_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   id                188533 non-null  int64   
 1   brand             188533 non-null  category
 2   model             188533 non-null  category
 3   model_year        188533 non-null  int64   
 4   milage            188533 non-null  int64   
 5   fuel_type         182669 non-null  category
 6   ext_col           188167 non-null  category
 7   int_col           184006 non-null  category
 8   accident          186081 non-null  category
 9   clean_title       167114 non-null  category
 10  price             188533 non-null  int64   
 11  transm_           188240 non-null  category
 12  hp                155274 non-null  float16 
 13  cyl               172766 non-null  category
 14  liter             181763 non-null  float16 
 15  luxurious_brands  188533 non-null  category
 16  to

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_for_model = train_df_for_model.groupby('price').sample(frac=1, random_state= 1).reset_index(drop = True)
X = df_for_model.drop(columns=['price'], axis =1)
y = df_for_model['price']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state = 8
)

In [ ]:
def objective_lgbm(trial):

    lgbm_params = {
    'n_estimators': trial.suggest_int('n_estimators', 800, 1500),
    'num_leaves': trial.suggest_int('num_leaves', 75, 200),
    'max_depth': trial.suggest_int('max_depth', 12, 30),
    'cat_smooth': trial.suggest_int('cat_smooth', 20, 120),
    'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.02),
    'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
    'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 0.8),
    'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-4, 1.0),
    'min_child_weight': trial.suggest_int('min_child_weight', 5, 70),
    'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-10, 1e-3),
    'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-10, 1e-2),
    #'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 1.0, 12.0),
    'max_bin': trial.suggest_int('max_bin', 200, 1000),
    }


    # Fit the model
    model_lgbm = LGBMRegressor(
        **lgbm_params,
        random_state=42,
        loss_function="RMSE",
        verbose=-1
    )

    model_lgbm.fit(X_train, y_train,
                   eval_set=[(X_test, y_test)],
                   callbacks=[lgb.early_stopping(3)]
            )

    # Predict and calculate mean squared error score
    y_pred = model_lgbm.predict(X_test)

    return mean_squared_error(y_test, y_pred, squared = False)

In [ ]:
pip install optuna

In [ ]:
!pip install lightgbm
import optuna
from lightgbm import LGBMRegressor #Import LGBMRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error


study_lgbm = optuna.create_study(
    study_name="LGBM_used_car",
    #sampler=TPESampler(),
    direction="minimize"
)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study_lgbm.optimize(objective_lgbm, n_trials=50, show_progress_bar=True)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


  0%|          | 0/50 [00:00<?, ?it/s]

<ipython-input-24-144c53117513>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.02),
<ipython-input-24-144c53117513>:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
<ipython-input-24-144c53117513>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 0.8),
<ipython-input-24-144c53117513>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This

Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[211]	valid_0's l2: 5.03969e+09


<ipython-input-24-144c53117513>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.02),
<ipython-input-24-144c53117513>:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
<ipython-input-24-144c53117513>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 0.8),
<ipython-input-24-144c53117513>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This

...

Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[245]	valid_0's l2: 5.00752e+09


<ipython-input-24-144c53117513>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.02),
<ipython-input-24-144c53117513>:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
<ipython-input-24-144c53117513>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 0.8),
<ipython-input-24-144c53117513>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This

Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[217]	valid_0's l2: 4.99608e+09


<ipython-input-24-144c53117513>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.02),
<ipython-input-24-144c53117513>:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
<ipython-input-24-144c53117513>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 0.8),
<ipython-input-24-144c53117513>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This

Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[381]	valid_0's l2: 5.00821e+09


In [ ]:


print("Best trial:", study_lgbm.best_trial)

Best trial: FrozenTrial(number=33, state=TrialState.COMPLETE, values=[70606.92351475633], datetime_start=datetime.datetime(2024, 9, 13, 8, 8, 57, 577914), datetime_complete=datetime.datetime(2024, 9, 13, 8, 9, 22, 265458), params={'n_estimators': 989, 'num_leaves': 130, 'max_depth': 18, 'cat_smooth': 117, 'learning_rate': 0.005283679873939653, 'subsample': 0.9449508385097357, 'colsample_bytree': 0.5950911935136813, 'min_split_gain': 0.18516595863848312, 'min_child_weight': 59, 'lambda_l2': 2.536235533805147e-09, 'lambda_l1': 0.00022310204558385324, 'max_bin': 588}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=1500, log=False, low=800, step=1), 'num_leaves': IntDistribution(high=200, log=False, low=75, step=1), 'max_depth': IntDistribution(high=30, log=False, low=12, step=1), 'cat_smooth': IntDistribution(high=120, log=False, low=20, step=1), 'learning_rate': FloatDistribution(high=0.02, log=True, low=0.001, step=None), 'sub

In [ ]:
print("Best parameters:", study_lgbm.best_params)

Best parameters: {'n_estimators': 989, 'num_leaves': 130, 'max_depth': 18, 'cat_smooth': 117, 'learning_rate': 0.005283679873939653, 'subsample': 0.9449508385097357, 'colsample_bytree': 0.5950911935136813, 'min_split_gain': 0.18516595863848312, 'min_child_weight': 59, 'lambda_l2': 2.536235533805147e-09, 'lambda_l1': 0.00022310204558385324, 'max_bin': 588}


In [ ]:
fig = optuna.visualization.plot_optimization_history(study_lgbm)
fig.update_layout(
    font_family="Courier New",
    font_size=12,
    title_font_family="Times New Roman",
    title_font_color="steelblue",
    legend_title_font_color="black",
    plot_bgcolor='#e9eff6',
    paper_bgcolor='#f5fbf5',
)
fig.update_xaxes(color = "darkgreen")
fig.update_traces(marker_color='green')

In [ ]:
lgbm_final = LGBMRegressor(**study_lgbm.best_params, importance_type="gain", verbose=-1)
lgbm_final.fit(X_train, y_train)

y_pred = lgbm_final.predict(X_test)
print("Root Mean squared error: ", mean_squared_error(y_test, y_pred, squared = False))

Root Mean squared error:  70664.75092812857


In [ ]:
sample_submission = pd.read_csv('/content/sample_submission.csv')

In [ ]:
# Only assign the predicted values to a portion of the 'sample_submission' DataFrame
sample_submission["price"][:len(y_pred)] =  y_pred
sample_submission.to_csv('submission.csv',index=False)
sample_submission.to_csv('D:\acer\Documents\playground-series-s4e9\Submission.csv', index=False)

<ipython-input-35-4457af8fa9c3>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

